In [1]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import HoverTool
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
from bokeh.models import DatetimeTickFormatter, NumeralTickFormatter
from bokeh.palettes import Category20
import matplotlib as plt
from math import pi
from bokeh.transform import cumsum
output_notebook()

Loading BokehJS ...

In [2]:
Data = pd.date_range(start ='2019-11-22', end ='2020-12-22', freq = 'm')

In [3]:
rotativo_dia = [np.random.uniform(400, 600) for i in range(len(Data))]
valor_inad = [np.random.uniform(50, 100) for i in range(len(Data))]
valor_pre_inad = [np.random.uniform(300, 120) for i in range(len(Data))]
carteira_debito = [np.random.uniform(700, 900) for i in range(len(Data))]

In [4]:
data = dict(Data = Data, 
           carteira_debito = carteira_debito,
           rotativo_dia = rotativo_dia, 
           valor_pre_inad = valor_pre_inad,
           valor_inad = valor_inad)

In [5]:
data['acumulado_1'] = [(i + j ) for i, j in zip(data['carteira_debito'],
                                                data['rotativo_dia'])]

data['acumulado_2'] = [(i + j + k ) for i, j, k in zip(data['carteira_debito'],
                                                        data['rotativo_dia'],
                                                        data['valor_pre_inad'])]

data['acumulado_3'] = [(i + j + k + l) for i, j, k, l in zip(data['carteira_debito'],
                                                             data['rotativo_dia'],
                                                             data['valor_pre_inad'],
                                                             data['valor_inad'])]

In [6]:
def transform(i, j):
    a = i/j
    return f'{(1 - a)*100:.2f}%'

In [7]:
data['carteira_debito_var'] = [0]
data['acumulado_1_var'] = [0]
data['acumulado_2_var'] = [0]
data['acumulado_3_var'] = [0]
data['carteira_debito_var'] = [0]
data['valor_inad_var'] = [0]
data['valor_pre_inad_var'] = [0]
data['rotativo_dia_var'] = [0]

[(data['carteira_debito_var'].append(transform(data['carteira_debito'][i], data['carteira_debito'][i+1])),
 data['acumulado_1_var'].append(transform(data['acumulado_1'][i], data['acumulado_1'][i+1])),
 data['acumulado_2_var'].append(transform(data['acumulado_2'][i], data['acumulado_2'][i+1])),
 data['acumulado_3_var'].append(transform(data['acumulado_3'][i], data['acumulado_3'][i+1])), 
 data['rotativo_dia_var'].append(transform(data['rotativo_dia'][i], data['rotativo_dia'][i+1])), 
 data['valor_inad_var'].append(transform(data['valor_inad'][i], data['valor_inad'][i+1])), 
 data['valor_pre_inad_var'].append(transform(data['valor_pre_inad'][i], data['valor_pre_inad'][i+1])))
 for i in range(len(Data)-1)];

In [8]:
data['acumulado_1_str'] = [f'R$ {(i + j):.2f}'.replace('.', ',') for i,j in zip(data['carteira_debito'],
                                                                                data['rotativo_dia'])]

data['acumulado_2_str'] = [f'R$ {(i + j + k):.2f}'.replace('.', ',') for i,j,k in zip(data['carteira_debito'],
                                                                               data['rotativo_dia'], 
                                                                              data['valor_pre_inad'])]

data['acumulado_3_str'] = [f'R$ {(i + j + k + l):.2f}'.replace('.', ',') for i,j,k,l in zip(data['carteira_debito'],
                                                                                            data['rotativo_dia'], 
                                                                                            data['valor_pre_inad'], 
                                                                                            data['valor_inad'])]

# Gráfico Áreas empilhadas

In [25]:
source = ColumnDataSource(data=data)

p = figure(width=1200, height=500, x_axis_type="datetime")

In [26]:
p.varea_stack(['carteira_debito', 'rotativo_dia', 'valor_pre_inad', 'valor_inad'],
              x='Data', color=("navy", "lightblue", 'lemonchiffon', 'red'), source=source)

p.yaxis[0].formatter = NumeralTickFormatter(format="R$0,00")
p.xaxis[0].formatter = DatetimeTickFormatter(months="%b %Y")


In [27]:
r1 = p.circle('Data', 'carteira_debito', source=source, size=8, color='navy', line_color = 'black')
p.line(Data, carteira_debito ,color="black", line_width=2)
p.add_tools(HoverTool(renderers=[r1], tooltips=[('carteira_debito', '@carteira_debito'),
                                                ('carteira_debito_var', '@carteira_debito_var')]))

r2 = p.circle('Data', 'acumulado_1', source=source, size=8, color='lightblue', line_color = 'black')
p.line(Data, data['acumulado_1'] ,color="black", line_width=2)
p.add_tools(HoverTool(renderers=[r2], tooltips=[('rotativo_dia', '@rotativo_dia'), 
                                               ('rotativo_dia_var', '@rotativo_dia_var'),
                                                ('acumulado_1_str', '@acumulado_1_str'), 
                                               ('acumulado_1_var', '@acumulado_1_var')]))

r3 = p.circle('Data', 'acumulado_2', source=source, size=8, color='lemonchiffon', line_color = 'black')
p.line(Data, data['acumulado_2'] ,color="black", line_width=2)
p.add_tools(HoverTool(renderers=[r3], tooltips=[('valor_pre_inad', '@valor_pre_inad'), 
                                               ('valor_pre_inad_var', '@valor_pre_inad_var'),
                                                ('acumulado_2_str', '@acumulado_2_str'), 
                                               ('acumulado_2_var', '@acumulado_2_var')]))

r4 = p.circle('Data', 'acumulado_3', source=source, size=8, color='red', line_color = 'black')
p.line(Data, data['acumulado_3'] ,color="black", line_width=2)
p.add_tools(HoverTool(renderers=[r4], tooltips=[('valor_inad', '@valor_inad'), 
                                               ('valor_inad_var', '@valor_inad_var'),
                                                ('acumulado_3_str', '@acumulado_3_str'), 
                                               ('acumulado_3_var', '@acumulado_3_var')]))

labels1 = LabelSet(x='Data', y='carteira_debito', text='carteira_debito', level='glyph', text_color='navy',
                   border_line_color='black', x_offset=5, y_offset=5, source=source)
p.add_layout(labels1)

labels2 = LabelSet(x='Data', y='acumulado_1', text='acumulado_2_str', level='glyph', text_color=,
                   border_line_color='black',x_offset=5, y_offset=5, source=source)
p.add_layout(labels2)

labels3= LabelSet(x='Data', y='acumulado_2', text='acumulado_2_str', level='glyph',text_color=,
                  border_line_color='black',x_offset=5, y_offset=5, source=source)
p.add_layout(labels3)

labels4 = LabelSet(x='Data', y='acumulado_3', text='acumulado_3_str', level='glyph',text_color=,
                   border_line_color='black',x_offset=5, y_offset=5, source=source)
p.add_layout(labels4)


show(p)

# Gráfico linha

In [ ]:
def line(string):
    source = ColumnDataSource(data=data)

    p = figure(width=800, height=200, x_axis_type="datetime")

    p.xaxis[0].formatter = DatetimeTickFormatter(months="%b %Y")
    p.yaxis[0].formatter = NumeralTickFormatter(format="R$0,00")

    r = p.circle('Data', string, source=source, size=8, color='navy', line_color = 'black')
    p.line('Data', string ,color="black", line_width=2, source=source)
    p.add_tools(HoverTool(renderers=[r], tooltips=[('string', f'@{string}'),
                                                   (f'{string}_var', f'@{string}_var')]))
    show(p)

In [ ]:
line('carteira_debito')

In [ ]:
line('rotativo_dia')

In [ ]:
line('valor_pre_inad')

In [ ]:
line('valor_inad')

# Gráfico pizza

In [ ]:
lista_test = [carteira_debito[-1], rotativo_dia[-1], valor_pre_inad[-1], valor_inad[-1]]

def transform(i,j):
    a = (i/j)*100
    string = f'{a:.2f}%'
    return string

lista_perc =  [transform(i,sum(lista_test)) for i in lista_test]    

lista_angle = [i/sum(lista_test)*2*pi for i in lista_test]

color = ['blue', 'green', 'black','red']

lista_str = ['Débito', 'Parcelado em dia', 'Pré inadimplência', 'Inadimplência']

data = dict(valor=lista_test,
           angle=lista_angle,
           color=color,
           Percentual = lista_perc,
           Tipo = lista_str)

source = ColumnDataSource(data=data)

p = figure(plot_height=350)

r = p.annular_wedge(x=0, y=1,  inner_radius=0.15, outer_radius=0.25, direction="anticlock",
                start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'), source=source,
                    fill_color='color', line_color="white")
p.add_tools(HoverTool(renderers=[r], tooltips=[('Tipo', '@Tipo'),
                                               ('valor', '@valor'),
                                               ('Percentual', '@Percentual')]))
p.background_fill_alpha = 0

show(p)